In [ ]:
open("D:/federalistpapers.txt")


In [ ]:
text = read("C:/Users/ADMIN/OneDrive/Documents/GitHub/csds133-s23-2/federalistpapers.txt", String)
essays = split(text, "Federalist No.")
println(essays)